In [1]:
import pandas as pd
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [2]:
fly = pd.read_csv('aviation_accident.csv')

In [3]:
fly 

,Unnamed: 0,Event.Date,Injury.Severity,Aircraft.damage,Make,Number.of.Engines,Engine.Type,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight
0,0,1948-10-24,Fatal(2),Destroyed,Stinson,1.0,Reciprocating,2.0,0.0,0.0,0.0,UNK,Cruise
1,1,1962-07-19,Fatal(4),Destroyed,Piper,1.0,Reciprocating,4.0,0.0,0.0,0.0,UNK,Unknown
2,2,1974-08-30,Fatal(3),Destroyed,Cessna,1.0,Reciprocating,3.0,NaN,NaN,NaN,IMC,Cruise
3,3,1977-06-19,Fatal(2),Destroyed,Rockwell,1.0,Reciprocating,2.0,0.0,0.0,0.0,IMC,Cruise
4,4,1979-08-02,Fatal(1),Destroyed,Cessna,1.0,unknown,1.0,2.0,NaN,0.0,VMC,Approach
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84171,88869,2022-12-13,Non-Fatal,Substantial,PIPER,2.0,unknown,0.0,0.0,0.0,1.0,unknown,unknown
84172,88873,2022-12-14,Non-Fatal,Substantial,CIRRUS DESIGN CORP,1.0,unknown,0.0,0.0,0.0,1.0,VMC,unknown
84173,88876,2022-12-15,Non-Fatal,Substantial,SWEARINGEN,2.0,unknown,0.0,0.0,0.0,1.0,unknown,unknown
84174,88877,2022-12-16,Minor,Substantial,CESSNA,1.0,unknown,0.0,1.0,0.0,0.0,VMC,unknown


In [4]:
fly[['Injury.Severity', 'Aircraft.damage', 'Make', 'Number.of.Engines', 'Engine.Type', 'Weather.Condition', 'Broad.phase.of.flight']].nunique() # sprawdzam liczbę poszczególnych wartości występujących w kolumnach 

Injury.Severity            77
Aircraft.damage             5
Make                     8043
Number.of.Engines           7
Engine.Type                14
Weather.Condition           5
Broad.phase.of.flight      13
dtype: int64

In [6]:
fly['Make'].value_counts().head(10) # chcę wydobyć tylko te marki, których ilość powtarza się więcej niż 1000 razy 

Cessna     21860
Piper      11857
CESSNA      4525
Beech       4237
PIPER       2652
Bell        2025
Boeing      1227
Grumman     1089
Mooney      1081
BEECH        954
Name: Make, dtype: int64

In [7]:
mask=fly.Make.map(fly.Make.value_counts())< 1000 # wydobywam wartości, które powtarzają się mniej niż 1000 razy

In [8]:
fly.Make = fly.Make.mask(mask, 'other') # zamieniam wartości, które powtarzają się mniej niż 1000 razy na 'other'

In [9]:
fly['Make'].value_counts().count() # uzyskuję liczbę marek 9, a 10 marka reprezentuje 'other'

10

In [10]:
fly['Injury.Severity'].value_counts().sort_values(ascending=False) # sprawdzam ile jest rodzjaów "Injury.Severity", chce aby powtarzające się Fatal(1,2, itd.) były przypisane do głównego Fatal

Non-Fatal     65640
Fatal(1)       5968
Fatal          4487
Fatal(2)       3536
Incident       1800
              ...  
Fatal(265)        1
Fatal(21)         1
Fatal(49)         1
Fatal(78)         1
Fatal(30)         1
Name: Injury.Severity, Length: 77, dtype: int64

In [11]:
fly["Injury.Severity"].mask(fly['Injury.Severity'].str.startswith('Fatal'), 'Fatal', inplace=True) # zastępuje wartości powtarzające się wartości Fatal(1, 2, itd) na Fatal

In [77]:
fly[['Broad.phase.of.flight']].value_counts() # w kolumnie "Broad.phase.of.flight"  są dwie wartości "unknown", chce je zmienić aby była tylko jedna wartość unknwon

Broad.phase.of.flight
unknown                  23537
Landing                  15278
Takeoff                  12418
Cruise                   10190
Maneuvering               8092
Approach                  6470
Climb                     2008
Taxi                      1941
Descent                   1852
Go-around                 1352
Standing                   921
Other                      117
Unknown                      0
dtype: int64

In [78]:
fly["Broad.phase.of.flight"].mask(fly['Broad.phase.of.flight'].str.startswith('Unknown'), 'unknown', inplace=True) # zamieniam wartość "Unknown" na "unknown"

In [79]:
fly[["Broad.phase.of.flight"]].value_counts()

Broad.phase.of.flight
unknown                  23537
Landing                  15278
Takeoff                  12418
Cruise                   10190
Maneuvering               8092
Approach                  6470
Climb                     2008
Taxi                      1941
Descent                   1852
Go-around                 1352
Standing                   921
Other                      117
Unknown                      0
dtype: int64

In [81]:
fly[['Aircraft.damage']].value_counts() # w kolumnie "Aircraft.damage"  są dwie wartości "unknown", chce je zmienić aby była tylko jedna wartość unknwon

Aircraft.damage
Substantial        62599
Destroyed          17487
Minor               2266
unknown             1767
Unknown               57
dtype: int64

In [82]:
fly["Aircraft.damage"].mask(fly['Aircraft.damage'].str.startswith('Unknown'), 'unknown', inplace=True) # zamieniam wartość "Unknown" na "unknown"

In [83]:
fly[["Aircraft.damage"]].value_counts()

Aircraft.damage
Substantial        62599
Destroyed          17487
Minor               2266
unknown             1824
Unknown                0
dtype: int64

In [89]:
fly[['Engine.Type']].value_counts() # w kolumnie "Engine.Type"  są trzy wartości "unknown": unknown, UNK, Unknown, oraz dwie wartości "None" chce je zmienić aby było tylko po jednej odpowaiadającej wartości 

Engine.Type    
Reciprocating      69366
unknown             4934
Turbo Shaft         3577
Turbo Prop          3335
Turbo Fan           2250
Turbo Jet            680
None                  21
Electric               8
LR                     2
Geared Turbofan        1
Hybrid Rocket          1
UNK                    1
NONE                   0
Unknown                0
dtype: int64

In [116]:
fly["Engine.Type"].mask(fly['Engine.Type'].str.startswith('Unknown'), 'unknown', inplace=True)
fly["Engine.Type"].mask(fly['Engine.Type'].str.startswith('NONE'), 'None', inplace=True) # zamieniam na jednolite wartości w kolumnie
fly["Engine.Type"].mask(fly['Engine.Type'].str.startswith('UNK'), 'unknown', inplace=True)

In [117]:
fly[['Engine.Type']].value_counts()

Engine.Type    
Reciprocating      69366
unknown             4935
Turbo Shaft         3577
Turbo Prop          3335
Turbo Fan           2250
Turbo Jet            680
None                  21
Electric               8
LR                     2
Geared Turbofan        1
Hybrid Rocket          1
NONE                   0
UNK                    0
Unknown                0
dtype: int64

In [102]:
fly[['Weather.Condition']].value_counts() # w kolumnie "Weather.Condition"  są 3 wartości, przyporządkowane do "unknown": UNK, unk, unknown, chce je zmienić aby była tylko jedna odpowaidająca wartość  

Weather.Condition
VMC                  76103
IMC                   5797
unknown               1308
UNK                    737
Unk                    231
dtype: int64

In [103]:
fly["Weather.Condition"].mask(fly['Weather.Condition'].str.startswith('UNK'), 'unknown', inplace=True)
fly["Weather.Condition"].mask(fly['Weather.Condition'].str.startswith('Unk'), 'unknown', inplace=True) # zamieniam na jednolite wartości w kolumnie

In [104]:
fly[['Weather.Condition']].value_counts()

Weather.Condition
VMC                  76103
IMC                   5797
unknown               2276
UNK                      0
Unk                      0
dtype: int64

In [12]:
list_str_obj_cols = fly.columns[fly.dtypes == "object"].tolist()

In [13]:
for str_obj_col in list_str_obj_cols:
    fly[str_obj_col] = fly[str_obj_col].astype("category") # zastępuje kolumny 'object' na typ 'category'

In [14]:
fly.dtypes

Unnamed: 0                   int64
Event.Date                category
Injury.Severity           category
Aircraft.damage           category
Make                      category
Number.of.Engines          float64
Engine.Type               category
Total.Fatal.Injuries       float64
Total.Serious.Injuries     float64
Total.Minor.Injuries       float64
Total.Uninjured            float64
Weather.Condition         category
Broad.phase.of.flight     category
dtype: object

In [15]:
fly[['Injury.Severity', 'Aircraft.damage', 'Make', 'Number.of.Engines', 'Engine.Type', 'Weather.Condition', 'Broad.phase.of.flight']].nunique() # końcowa liczba unikalnych wartości w kolumnach do kodowania

Injury.Severity           6
Aircraft.damage           5
Make                     10
Number.of.Engines         7
Engine.Type              14
Weather.Condition         5
Broad.phase.of.flight    13
dtype: int64

# OneHotEncoding

In [64]:
from sklearn.compose import make_column_transformer

In [118]:
ohc = OneHotEncoder(sparse=False).fit(fly[['Injury.Severity','Aircraft.damage', 'Make', 'Number.of.Engines', 'Engine.Type', 'Weather.Condition', 'Broad.phase.of.flight']])

c:\Users\magda\anaconda3\envs\machine_learning\lib\site-packages\sklearn\preprocessing\_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [119]:
encoded = ohc.transform(fly[['Injury.Severity','Aircraft.damage', 'Make', 'Number.of.Engines', 'Engine.Type', 'Weather.Condition', 'Broad.phase.of.flight']])

In [120]:
encoded_df = pd.DataFrame(encoded, columns = ohc.get_feature_names_out())

In [121]:
encoded_df

,Injury.Severity_Fatal,Injury.Severity_Incident,Injury.Severity_Minor,Injury.Severity_Non-Fatal,Injury.Severity_Serious,Injury.Severity_Unavailable,Aircraft.damage_Destroyed,Aircraft.damage_Minor,Aircraft.damage_Substantial,Aircraft.damage_unknown,...,Broad.phase.of.flight_Cruise,Broad.phase.of.flight_Descent,Broad.phase.of.flight_Go-around,Broad.phase.of.flight_Landing,Broad.phase.of.flight_Maneuvering,Broad.phase.of.flight_Other,Broad.phase.of.flight_Standing,Broad.phase.of.flight_Takeoff,Broad.phase.of.flight_Taxi,Broad.phase.of.flight_unknown
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84171,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
84172,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
84173,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
84174,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [124]:
aviation_accident_encoded = fly.join(encoded_df )

In [126]:
aviation_accident_encoded = aviation_accident_encoded[["Event.Date", 'Injury.Severity_Fatal', 'Injury.Severity_Incident',
       'Injury.Severity_Minor', 'Injury.Severity_Non-Fatal',
       'Injury.Severity_Serious', 'Injury.Severity_Unavailable',
       'Aircraft.damage_Destroyed', 'Aircraft.damage_Minor',
       'Aircraft.damage_Substantial',
       'Aircraft.damage_unknown', 'Make_Beech', 'Make_Bell', 'Make_Boeing',
       'Make_CESSNA', 'Make_Cessna', 'Make_Grumman', 'Make_Mooney',
       'Make_PIPER', 'Make_Piper', 'Make_other', 'Number.of.Engines_0.0',
       'Number.of.Engines_1.0', 'Number.of.Engines_2.0',
       'Number.of.Engines_3.0', 'Number.of.Engines_4.0',
       'Number.of.Engines_6.0', 'Number.of.Engines_8.0',
       'Engine.Type_Electric', 'Engine.Type_Geared Turbofan',
       'Engine.Type_Hybrid Rocket', 'Engine.Type_LR', 
       'Engine.Type_None', 'Engine.Type_Reciprocating',
       'Engine.Type_Turbo Fan', 'Engine.Type_Turbo Jet',
       'Engine.Type_Turbo Prop', 'Engine.Type_Turbo Shaft', 
       'Engine.Type_unknown', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured', 'Weather.Condition_IMC',
       'Weather.Condition_VMC', 'Weather.Condition_unknown',
       'Broad.phase.of.flight_Approach', 'Broad.phase.of.flight_Climb',
       'Broad.phase.of.flight_Cruise', 'Broad.phase.of.flight_Descent',
       'Broad.phase.of.flight_Go-around', 'Broad.phase.of.flight_Landing',
       'Broad.phase.of.flight_Maneuvering', 'Broad.phase.of.flight_Other',
       'Broad.phase.of.flight_Standing', 'Broad.phase.of.flight_Takeoff',
       'Broad.phase.of.flight_Taxi',
       'Broad.phase.of.flight_unknown' ]]

In [127]:
aviation_accident_encoded.columns

Index(['Event.Date', 'Injury.Severity_Fatal', 'Injury.Severity_Incident',
       'Injury.Severity_Minor', 'Injury.Severity_Non-Fatal',
       'Injury.Severity_Serious', 'Injury.Severity_Unavailable',
       'Aircraft.damage_Destroyed', 'Aircraft.damage_Minor',
       'Aircraft.damage_Substantial', 'Aircraft.damage_unknown', 'Make_Beech',
       'Make_Bell', 'Make_Boeing', 'Make_CESSNA', 'Make_Cessna',
       'Make_Grumman', 'Make_Mooney', 'Make_PIPER', 'Make_Piper', 'Make_other',
       'Number.of.Engines_0.0', 'Number.of.Engines_1.0',
       'Number.of.Engines_2.0', 'Number.of.Engines_3.0',
       'Number.of.Engines_4.0', 'Number.of.Engines_6.0',
       'Number.of.Engines_8.0', 'Engine.Type_Electric',
       'Engine.Type_Geared Turbofan', 'Engine.Type_Hybrid Rocket',
       'Engine.Type_LR', 'Engine.Type_None', 'Engine.Type_Reciprocating',
       'Engine.Type_Turbo Fan', 'Engine.Type_Turbo Jet',
       'Engine.Type_Turbo Prop', 'Engine.Type_Turbo Shaft',
       'Engine.Type_unknown', 

In [128]:
aviation_accident_encoded.to_csv('Aviation_accident.encoded.csv') # zapisuję encodowany plik do csv. 